In [1]:
import scipy
import scipy.io as sio
import numpy as np
from PIL import Image
import math
import os
from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import PReLU
#utilities help us transform our data
from keras.utils import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
rows, cols = 48,48
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf

import cv2
from keras.models import load_model
from statistics import mode
#from utils import preprocess_input
import matplotlib.pyplot as plt
%matplotlib inline


Using TensorFlow backend.
/home/ramesh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
import cv2
from random import shuffle

class Data_Generator:
    def __init__(self, data_path, batch_size, val_split):
        """
        Initialize image paths and gender labels
        # Arguments: metadata_path
        """
        self.data = scipy.io.loadmat(data_path)
        self.img_paths = self.data['wiki']['full_path'][0][0][0]
        self.genders = self.data['wiki']['gender'][0][0][0]
        self.face_score = self.data['wiki']['face_score'][0][0][0]
        self.sec_face_score = self.data['wiki']['second_face_score'][0][0][0]
        self.batch_size = batch_size
        self.val_split = val_split
        self.load_keys()


    def load_keys(self):
        indices = []
        count_male, count_female = 0, 0
        n_m, n_f = 7000, 7000
        for i in range(len(self.img_paths)):

            if count_female == n_f and count_male == n_m:
                break            
            
            if not np.isinf(self.face_score[i]) and self.face_score[i] > 3.0:
                if np.isnan(self.sec_face_score[i]):
                    if not np.isnan(self.genders[i]):
                        gender = self.genders[i]
                        if gender == 0 and count_female < n_f:
                            count_female += 1
                            indices.append(i)
                        elif gender == 1 and count_male < n_m:
                            count_male += 1
                            indices.append(i)  
                               
        self.number_of_imgs = len(indices)

        self.val_size = int(self.number_of_imgs * self.val_split)
        self.train_size = self.number_of_imgs - self.val_size
        shuffle(indices)
        self.train_keys = indices[:self.train_size]
        self.val_keys = indices[self.train_size:]


    def load_data(self, is_train):
        
        """ load wiki dataset 
        # Returns: faces and genders
                face: shape (64, 64, 1)
                gender_labels: 0 for female and 1 for male
        """        
        while 1:
            faces = []
            gender_labels = []   

            shuffle(self.train_keys)
            keys = self.train_keys
            if not is_train:
                keys = self.val_keys

            for key in keys:
                img_path = self.img_paths[key][0]
                img = cv2.imread('./wiki_crop/' + img_path, 0)            

                faces.append(cv2.resize(img, (64, 64)))
                gender_labels.append(self.genders[key])

                if len(faces) == self.batch_size:
                    faces = np.expand_dims(faces,-1)
                    gender_labels = pd.get_dummies(gender_labels).as_matrix()
                    yield (faces, gender_labels)
                    faces = []
                    gender_labels = []

            if len(faces) == 0:
                continue
            faces = np.expand_dims(faces,-1)
            gender_labels = pd.get_dummies(gender_labels).as_matrix()
            yield (faces, gender_labels)






In [41]:
'''
class Data_Generator:
    
    def __init__(self, metadata_path, batch_size, val_split):

        self.data = scipy.io.loadmat(metadata_path)
        self.img_paths = self.data['wiki']['full_path'][0][0][0]
        self.genders = self.data['wiki']['gender'][0][0][0]
        self.batch_size = batch_size
        self.val_split = val_split  
        #self.sorted_gender = []
        self.face_score = self.data['wiki']['face_score'][0][0][0]
        self.second_face_score = self.data['wiki']['second_face_score'][0][0][0]
        self.load_keys()
        
        
    def load_keys(self):
        indices = []
        male_count_face,female_count_face = 0,0
        
        total_number_of_male_faces, total_number_of_female_faces = 6500,6500 
        
        for i in range(len(self.img_paths)):
            
            #data = self.face_score[i]

            # second face score
            data_second_face_score = self.second_face_score[i]

            # if face_score is infinity, image is completely black 
            if male_count_face == total_number_of_male_faces and female_count_face == total_number_of_female_faces:
                break
                
            

                # if face score is greater than 3 and second face score is nan then image is somewhat better 
                # Looks readable
            if (self.face_score[i] > 3.00000000000 and (math.isnan(data_second_face_score) == True)):
                if (math.isinf(self.face_score[i]) == False):
                    if (math.isnan(self.genders[i]) == False):
                        #gender = self.genders[i]
                        if (self.genders[i] == 0) and (female_count_face < total_number_of_female_faces):
                            #if female_count_face < number_of_female_faces:
                            female_count_face = female_count_face + 1
                            indices.append(i)
                        elif (self.genders[i] == 1) and (male_count_face < total_number_of_male_faces):
                            
                            male_count_face = male_count_face + 1
                            indices.append(i)
                        
        print('Males: %d'%female_count_face)
        print('Females: %d'%male_count_face)
        
        self.number_of_imgs = len(indices)
        self.val_size = int(self.number_of_imgs * self.val_split)
        self.train_size = self.number_of_imgs - self.val_size
        self.train_keys = indices[:self.train_size]
        self.val_keys = indices[self.train_size:]


    def load_data(self, is_train):
        count = 0
        while 1:
            faces = []
            gender_labels = []
            #female_list = []
            count = count + 1
            shuffle(self.train_keys)
            keys = self.train_keys
            if not is_train:
                keys = self.val_keys 

            for key in keys:
                img_path = self.img_paths[key][0]
                #location_of_image = './wiki_crop/' + img_path
                #img = cv2.imread('./wiki_crop/')
                img = cv2.imread('./wiki_crop/' + img_path, 0)
                faces.append(cv2.resize(img,(64,64)))
                
                gender_labels.append(self.genders[key])
                
                #print gender_labels                    
                #image = Image.open(location_of_image)
                
                #image_resized = image.resize((rows,cols))
                #image_gray = image_resized.convert('L')
                #plt.plot(image_gray)
                #face = np.asarray(image_gray)
                #faces.append(image_gray)
                    
                if len(faces) == self.batch_size:
                    #faces = np.asarray(faces)
                    faces = np.expand_dims(faces,-1)
                    gender_labels = pd.get_dummies(gender_labels).as_matrix()
                    
                    yield (faces, gender_labels)
                    faces = []
                    gender_labels = []
            if len(faces) == 0:
                continue
            #faces = np.asarray(faces)
            faces = np.expand_dims(faces,-1)
            gender_labels = pd.get_dummies(gender_labels).as_matrix()
            yield (faces, gender_labels)
 '''               

In [45]:

data_path = './wiki_crop/wiki.mat'
model_save_path = './wiki_crop/simpler_Combined_CNN_1.hdf5'
num_classes = 2
image_size = (64, 64, 1)
batch_size = 150
num_epochs = 20
val_split = 0.1
data = Data_Generator(data_path, batch_size, val_split)
gen_train = data.load_data(True)
gen_val = data.load_data(False)
#print gender_la

Males: 7000
Females: 6580


In [46]:
model = CNN(image_size, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
                                        metrics=['accuracy'])
csv_logger = CSVLogger('training.log')
early_stop = EarlyStopping('val_acc', patience=200, verbose=1)
model_checkpoint = ModelCheckpoint(model_save_path,
                                    'val_acc', verbose=0,
                                    save_best_only=True)

model_callbacks = [early_stop, model_checkpoint, csv_logger]

#keras bug 
K.get_session().run(tf.global_variables_initializer())
model.fit_generator(gen_train, nb_epoch=num_epochs, verbose=1, 
                                    validation_data=gen_val,
                                    samples_per_epoch=data.train_size,
                                    nb_val_samples=data.val_size,
                                    callbacks=model_callbacks)

Epoch 1/20
12222/12222 [==============================] - 525s - loss: 0.7354 - acc: 0.6568 - val_loss: 0.6084 - val_acc: 0.6900
Epoch 2/20
12222/12222 [==============================] - 404s - loss: 0.5218 - acc: 0.7464 - val_loss: 0.5474 - val_acc: 0.7253
Epoch 3/20
12222/12222 [==============================] - 394s - loss: 0.4696 - acc: 0.7799 - val_loss: 0.4785 - val_acc: 0.7658
Epoch 4/20
12222/12222 [==============================] - 396s - loss: 0.4302 - acc: 0.8052 - val_loss: 0.4674 - val_acc: 0.7931
Epoch 5/20
12222/12222 [==============================] - 392s - loss: 0.3999 - acc: 0.8225 - val_loss: 0.4098 - val_acc: 0.8041
Epoch 6/20
12222/12222 [==============================] - 394s - loss: 0.3694 - acc: 0.8401 - val_loss: 0.3750 - val_acc: 0.8461
Epoch 7/20
12222/12222 [==============================] - 476s - loss: 0.3556 - acc: 0.8480 - val_loss: 0.3512 - val_acc: 0.8417
Epoch 8/20
12222/12222 [==============================] - 442s - loss: 0.3511 - acc: 0.8458 - val

In [9]:
def CNN(input_shape,num_classes):
    model = Sequential()

    model.add(Convolution2D(16, 7, 7, border_mode='same',
                            input_shape=input_shape))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(5, 5),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 5, 5, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(3, 3),strides=(2, 2), border_mode='same'))
    model.add(Dropout(.5))

    model.add(Flatten())
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(1028))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [47]:
def preprocess_image(images):
    
    images = images/255.0
    
    return images

In [ ]:
classification_model_path = './wiki_crop/simpler_Combined_CNN_1.hdf5'
detection_model_path = './wiki_crop/haarcascade_frontalface_default.xml'
frame_window = 10
gender_labels = {0:'Female',1:'Male'}

face_detection = cv2.CascadeClassifier(detection_model_path)
gender_classifier = load_model(classification_model_path)
gender_window = []
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.startWindowThread()
cv2.namedWindow('window_frame')
while True:
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x - (int)(0.2*w), y - (int)(0.3*h)),(x + (int)(1.2*w), y + (int)(1.2*h)),(255,0,0),2)
        face = gray[y - (int)(0.3*h) : y + (int)(1.2*h), x - (int)(0.2*w) : x + (int)(1.2*w)]
        try:
            face = cv2.resize(face, (64, 64))
        except:
            continue
        face = np.expand_dims(face,0)
        face = np.expand_dims(face,-1)
        #face = preprocess_input(face)
        #face = face - 143
        gender_arg = np.argmax(gender_classifier.predict(face))
        gender = gender_labels[gender_arg]
        gender_window.append(gender)

        if len(gender_window) >= frame_window:
            gender_window.pop(0)
        try:
            gender_mode = mode(gender_window)
        except:
            continue
        cv2.putText(gray,gender_mode,(x,y-30), font, .7,(255,0,0),1,cv2.LINE_AA)
    try:
        cv2.imshow('window_frame', gray)
    except:
        continue

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()